In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from scipy.stats import norm
plt.rcParams["figure.figsize"] = (5,4)

In [2]:
res_1 = 'results/dict/'
res_2 = '../temp/results/dict/'
file_name = [res_1 + f for f in os.listdir(res_1)] + [res_2 + f for f in os.listdir(res_2)]

In [4]:
clip = 2.0
folds = 5
lr = 0.5
epsilon = [0.5, 1.0, 3.0, 5.0, 7.0, 9.0]
noise_scale_bank = [21.75, 11.517, 4.3384, 2.83, 2.1743, 1.8079]
noise_scale_adult = [3.55, 1.98, 0.982, 0.794, 0.704, 0.648]

### Utils

In [5]:
def read_pickel(file):
    with open(file, 'rb') as f:
        res = pickle.load(f)
    return res

In [6]:
def disperate_impact(male_loader, female_loader, global_model, male_model, female_model, num_male, num_female, device = 'cpu'):
    global_model.to(device)
    male_model.to(device)
    female_model.to(device)

    glob_male_out = []
    glob_female_out = []
    male_outputs = []
    female_outputs = []

    global_model.eval()
    male_model.eval()
    female_model.eval()
    with torch.no_grad():

        for bi, d in enumerate(male_loader):
            
            features, _, ismale = d
            # print(ismale) 
            
            features = features.to(device, dtype=torch.float)

            glob_out = global_model(features)
            male_out = male_model(features)

            glob_out = torch.squeeze(glob_out, dim=-1)
            glob_out = glob_out.cpu().detach().numpy()
            glob_male_out.extend(glob_out)

            male_out = torch.squeeze(male_out, dim=-1)
            male_out = male_out.cpu().detach().numpy()
            male_outputs.extend(male_out)

        for bi, d in enumerate(female_loader):
            features, _, ismale = d
            # print(ismale)
            features = features.to(device, dtype=torch.float)

            glob_out = global_model(features)
            female_out = female_model(features)

            glob_out = torch.squeeze(glob_out, dim=-1)
            glob_out = glob_out.cpu().detach().numpy()
            glob_female_out.extend(glob_out)

            female_out = torch.squeeze(female_out, dim=-1)
            female_out = female_out.cpu().detach().numpy()
            female_outputs.extend(female_out)

    male_outputs = np.array(male_outputs)
    glob_male_out = np.array(glob_male_out)
    female_outputs = np.array(female_outputs)
    glob_female_out = np.array(glob_female_out)

    male_norm = np.sum(np.abs(male_outputs - glob_male_out))
    female_norm = np.sum(np.abs(female_outputs - glob_female_out))
    return male_norm / num_male, female_norm / num_female

In [7]:
def get_tilde_sigma(ns, clip, bz1, bz2):
    return ns*clip*np.sqrt(1/(bz1**2) + 1/(bz2**2))

def expected_bound(ns, clip, num_params, lr):
    return 2*lr*clip*np.sqrt(num_params) + lr*num_params*ns*np.sqrt(2)/np.sqrt(np.pi)

def worstcase_bound(ns, num_params, lr, alpha=0.05):
    return 2*lr*clip*np.sqrt(num_params) + lr*ns*np.sqrt(-2*np.log(alpha)*num_params + 2*num_params**2*np.log(2))

### Read Data

In [ ]:
df = pd.read_csv('Data/Bank/formated_bank.csv')
feature_cols = list(df.columns)
feature_cols.remove('y')
feature_cols.remove('z')
feature_cols.remove('label')
feature_cols.remove('is_train')
feature_cols.remove('intercept')
label = 'y'
train_df = df[df['is_train'] == 1].reset_index(drop=True).sample(frac = 1)
test_df = df[df['is_train'] == 0].reset_index(drop=True).sample(frac = 1)
male_df = test_df[test_df['z'] == 1]
female_df = test_df[test_df['z'] == 0]